In [1]:
import time

# Start the timer
start_time = time.time()
from queue import PriorityQueue
from collections import defaultdict

start = 'A'
end = 'D'
avoid_node = 'C'

# The graph is defined as a dictionary where each key represents a node and its value is another dictionary containing the neighboring nodes and their respective weights.
graph = {
  'A': {'B': 2, 'C': 3},
  'B': {'A': 2, 'D': 4},
  'C': {'A': 3, 'D': 1},
  'D': {'B': 4, 'C': 1},
}

def kruskal_mst_with_removed_node(graph, removed_node=avoid_node):
    # Initialize variables
    mst = []
    edges = PriorityQueue()
    parent = {}

    # Define a function to find the root of a node in the disjoint set
    def find_root(node):
        while parent[node] != node:
            parent[node] = parent[parent[node]]
            node = parent[node]
        return node

    # Initialize the parent dictionary and add all edges to the priority queue
    for node in graph:
        parent[node] = node
        for neighbor, weight in graph[node].items():
            if node != removed_node and neighbor != removed_node: # skip edge with removed node
                edges.put((weight, node, neighbor))

    # Loop through all edges and add them to the MST if they don't create a cycle
    while not edges.empty():
        weight, u, v = edges.get()
        root_u = find_root(u)
        root_v = find_root(v)
        if root_u != root_v:
            mst.append((u, v, weight))
            parent[root_u] = root_v

    # Sort the MST based on the order of the nodes in the edge tuple
    mst.sort(key=lambda x: (graph[x[0]][x[1]], x[0], x[1]))

    return mst

import heapq

def dijkstra_shortest_path(graph, start, end, removed_node=avoid_node):
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    queue = [(0, start)]
    prev_nodes = {node: None for node in graph}

    while queue:
        current_distance, current_node = heapq.heappop(queue)

        # Skip nodes that have been removed from the graph
        if current_node == removed_node:
            continue

        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            if neighbor == removed_node:
                continue
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                prev_nodes[neighbor] = current_node
                heapq.heappush(queue, (distance, neighbor))

    path = []
    node = end
    while node is not None:
        path.append(node)
        node = prev_nodes[node]

    path.reverse()

    return path if distances[end] != float('inf') else None


# Define a function to find an alternate route for a delayed train or closed track
def find_alternate_route(mst, origin, destination, closed_track, heuristic='shortest_distance'):
    # Initialize variables
    edges = PriorityQueue()
    path = []
    visited = set()
    stack = [origin]
    distances = {node: float('inf') for node in graph}  # distance from origin to each node
    distances[origin] = 0
    time = {node: float('inf') for node in graph}  # time from origin to each node
    time[origin] = 0
    parent = {}
    
    # Define a function to find the root of a node in the disjoint set
    def find_root(node):
        while parent[node] != node:
            parent[node] = parent[parent[node]]
            node = parent[node]
        return node

    # Initialize the parent dictionary and add all edges to the priority queue
    for node in graph:
        parent[node] = node
        for neighbor, weight in graph[node].items():
            edges.put((weight, node, neighbor))
    
    # Loop through all edges and add them to the MST if they don't create a cycle
    while not edges.empty():
        weight, u, v = edges.get()
        root_u = find_root(u)
        root_v = find_root(v)
        if root_u != root_v:
            mst.append((u, v, weight))
            parent[root_u] = root_v
    
    # Sort the MST based on the order of the nodes in the edge tuple
    mst.sort(key=lambda x: (graph[x[0]][x[1]], x[0], x[1]))
    
    # This function performs a depth-first search on a graph to find a path from a starting node to a destination node. It uses a minimum spanning tree (mst) to traverse the graph.
    def dfs(node):
        
        visited.add(node)
        path.append(node)
        if node == destination:
            return True
        for u, v, weight in mst:
            if u == node and v not in visited and v != closed_track:
                if heuristic == 'shortest_distance':
                    if distances[node] + weight < distances[v]:
                        distances[v] = distances[node] + weight
                        parent[v] = node
                    if dfs(v):
                        return True
                elif heuristic == 'shortest_time':
                    if time[node] + weight < time[v]:
                        time[v] = time[node] + weight
                        parent[v] = node
                    if dfs(v):
                        return True
        # True if the destination node is found, False otherwise.
        path.pop()
        return False
    
    # Call the DFS function to find the path
    dfs(origin)
    
    # Check if the path is valid
    if not path:
        print("No valid path found.")
    elif path[-1] != destination:
        print("No valid path found.")
    else:
        # Given a path and a destination, print out the alternate route if it exists.
        print("Alternate Route:")
        for i in range(len(path)-1):
            u, v = path[i], path[i+1]
            for edge in mst:
                if edge[0] == u and edge[1] == v:
                    print(f"{u} - {v} : {edge[2]}")
                    break

# Find the minimum spanning tree using Kruskal's algorithm
mst = kruskal_mst_with_removed_node(graph,avoid_node)
print("Minimum Spanning Tree:")
for edge in mst:
    print(f"{edge[0]} - {edge[1]} : {edge[2]}")

shortest_path = dijkstra_shortest_path(graph, start, end, avoid_node)
if shortest_path is not None:
    print(f"Shortest path from {start} to {end} with {avoid_node} removed: {' -> '.join(shortest_path)}")
else:
    print(f"No path found from {start} to {end} with {avoid_node} removed.")

# Find an alternate route using DFS
alternate_route = find_alternate_route(mst, start, end, closed_track=avoid_node, heuristic='shortest_distance')

# Define a function to find all paths in a graph that do not contain a particular node
def find_paths(graph, start, end, avoid_node):
    paths = []
    visited = set()

    # Define a helper function to recursively find paths
    def dfs(node, path):
        if node == end:
            paths.append(path)
        else:
            visited.add(node)
            for neighbor in graph[node]:
                if neighbor not in visited and neighbor != avoid_node:
                    dfs(neighbor, path + [neighbor])
            visited.remove(node)

    # Call the helper function to find all paths from the start node to the end node
    dfs(start, [start])
    return paths
paths = find_paths(graph, start, end, avoid_node)
for path in paths:
    print(path)


# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"The code block took {elapsed_time} seconds to run.")

Minimum Spanning Tree:
A - B : 2
B - D : 4
Shortest path from A to D with C removed: A -> B -> D
Alternate Route:
A - B : 2
B - D : 4
['A', 'B', 'D']
The code block took 0.012480974197387695 seconds to run.
